# `CLOUD.jl` - 2D linear advection example

In [19]:
using CLOUD, OrdinaryDiffEq

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423


Define the physical problem

In [20]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

initial_data = InitialDataSine(A,k)
conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)));

Set discretization parameters

In [21]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(StandardMapping(),
    LaxFriedrichsNumericalFlux())
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [22]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [23]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            421ms / 460.7%          57.4MiB / 1779.4%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          40.8k    1.09s   56.3%  26.8μs    510MiB   49.9%  12.8KiB
   facet terms          40.8k    528ms   27.2%  12.9μs    223MiB   21.9%  5.60KiB
   volume terms         40.8k    397ms   20.5%  9.72μs    224MiB   21.9%  5.62KiB
   mass matrix solve    40.8k   42.3ms    2.2%  1.04μs   19.9MiB    1.9%     512B
 extrap solution        40.8k    591ms   30.5%  14.5μs    198MiB   19.3%  4.96KiB
 eval flux              40.8k    159ms    8.2%  3.89μs   77.7MiB    7.6%  1.95KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [24]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00068216127441316
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085946192285201e-8][-7.840950111415168e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497973419588697][-2.025391697368928e-5]


## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [25]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

"results/advection_2d_dgmulti/"

Run the solver

In [26]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         86.4μs / 1314234.4%      10.6KiB / 10600162.7%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.6k    676ms   59.6%  8.29μs    471MiB   43.1%  5.91KiB
   volume terms         81.6k    242ms   21.4%  2.97μs    212MiB   19.4%  2.66KiB
   mass matrix solve    81.6k    193ms   17.0%  2.36μs    186MiB   17.0%  2.33KiB
   facet terms          81.6k    132ms   11.6%  1.62μs   25.3MiB    2.3%     325B
 eval flux              81.6k    146ms   12.8%  1.78μs   69.8MiB    6.4%     896B
 gather ext state       81.6k    118ms   10.4%  1.44μs    350MiB   32.0%  4.39KiB
 extrap solutio

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [27]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034594895345242908
Conservation (initial/final/diff):
[-1.4530229013823975e-7][-1.4530229063480435e-7][-4.96564594998361e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485531367220426][-0.0001394365068460024]
